<a href="https://colab.research.google.com/github/artem-barsov/Numerical-Methods/blob/main/lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from math import log10, atan, pi, sin, cos, sqrt

#### Форматирование

In [ ]:
def get_script(s, script='sup'):
    if script == 'sup':
        return s.translate(s.maketrans('0123456789', '⁰¹²³⁴⁵⁶⁷⁸⁹'))
    if script == 'sub':
        return s.translate(s.maketrans('0123456789', '₀₁₂₃₄₅₆₇₈₉'))
    return s

In [ ]:
def polynom2str(a, x = 'x', script='sup'):
    s = ''
    frm = 0
    while a[frm]==0: frm += 1
    first_done = False
    for i in range(frm, len(a)):
        if a[i] == 0: continue
        deg = len(a) - i - 1
        if first_done:
            s += ('+ ' if a[i] > 0 else '- ')
        elif a[i] < 0:
            s += '-'
        first_done = True
        if abs(a[i]) != 1 or deg == 0:
            s += '%g'%abs(a[i])
        if script == 'sup':
            if deg > 0:
                s += x
                if deg != 1:
                    s += get_script(str(deg), script)
                s += ' '
        else:
            s += x + get_script(str(i+1), script) + ' '
    return s if s!='' else '0'

In [ ]:
def print_system(a, b):
    for i in range(len(a)):
        print('{', polynom2str(a[i], script='sub'), '=', b[i])

In [ ]:
def print_x_solution(x, sym = 'x'):
    for i in range(1, len(x)+1):
        print(f'{sym}{get_script(str(i), "sub")} = %g;' % round(x[i-1], 3), end='  ')
    print()

In [ ]:
def print_vec_solution(x, sym = 'x'):
    for i in range(1, len(x)+1):
        print(f'{sym}{get_script(str(i), "sub")} = {x[i-1]};')
    print()

In [ ]:
def print_comp_solution(x, sym = 'x'):
    for i in range(1, len(x)+1):
        s = str(round(x[i-1][0], 4))
        if x[i-1][1] != 0:
            if x[i-1][1] < 0: s += ' - ' 
            else: s += ' + '
            s += str(round(abs(x[i-1][1]), 4)) + ' i'
        print(f'{sym}{get_script(str(i), "sub")} = {s};')
    print()

#### 1. Реализовать алгоритм LU -  разложения матриц (с выбором главного элемента) в виде программы. Используя разработанное программное обеспечение, решить систему линейных алгебраических уравнений (СЛАУ). Для матрицы СЛАУ вычислить определитель и обратную матрицу.

In [ ]:
class LU:
    EPS = 1e-6

    def __init__(self, U):
        L = np.eye(len(U), dtype=float)
        isDetNeg = False
        permut = np.array(range(len(U)))
        for i in range(len(U)):
            max_idx = i
            for j in range(i + 1, len(U)):
                if abs(U[max_idx][i]) < abs(U[j][i]):
                    max_idx = j
            if max_idx != i:
                U[[i, max_idx]] = U[[max_idx, i]]
                L[[i, max_idx]] = L[[max_idx, i]]
                L[:, [i, max_idx]] = L[:, [max_idx, i]]
                isDetNeg = not isDetNeg
                permut[[i, max_idx]] = permut[[max_idx, i]]
            if abs(U[i][i]) < self.EPS: continue
            for j in range(i + 1, len(U)):
                mu = U[j][i] / U[i][i]
                L[j][i] = mu
                for k in range(len(U)):
                    U[j][k] -= mu * U[i][k]
        det = U.diagonal().prod()
        if isDetNeg: det = -det
        self._permut = permut
        self.L = L
        self.U = U
        self.det = det

    def solve(self, b):
        b = np.array([ b[pi] for pi in self._permut ], dtype=float)
        z = np.array([0] * len(b), dtype=float)
        for i in range(len(b)):
            z[i] = b[i]
            for j in range(i):
                z[i] -= self.L[i, j] * z[j]
        x = np.array([0] * len(b), dtype=float)
        for i in range(len(b)-1, -1, -1):
            if abs(self.U[i, i]) < self.EPS: continue
            x[i] = z[i]
            for j in range(len(b)-1, i, -1):
                x[i] -= x[j] * self.U[i, j]
            x[i] /= self.U[i, i]
        return x

    def inverse(self):
        n = len(self.L)
        ret = np.matrix([[0] * n] * n, dtype=float)
        for i in range(n):
            b = np.array([0] * n, dtype=float)
            b[i] = 1
            ret[:, i] = np.matrix(self.solve(b)).T
        return ret

In [ ]:
A = np.array([[ 1,  2, -2,  6],
              [-3, -5, 14, 13],
              [ 1,  2, -2, -2],
              [-2, -4,  5, 10]], dtype=float)
b = np.array([24, 41, 0, 20], dtype=float)
print_system(A, b)

{ x₁ + 2x₂ - 2x₃ + 6x₄  = 24.0
{ -3x₁ - 5x₂ + 14x₃ + 13x₄  = 41.0
{ x₁ + 2x₂ - 2x₃ - 2x₄  = 0.0
{ -2x₁ - 4x₂ + 5x₃ + 10x₄  = 20.0


In [ ]:
lu = LU(A)
print('Решение системы: ', end='')
print_x_solution(lu.solve(b))
print('Определитель матрицы:', round(lu.det, 3))
print('Обратная матрица:\n', lu.inverse())

Решение системы: x₁ = 2;  x₂ = 4;  x₃ = 2;  x₄ = 3;  
Определитель матрицы: 8.0
Обратная матрица:
 [[-11.5    -2.     42.5    18.   ]
 [  5.125   1.    -18.125  -8.   ]
 [ -0.75    0.      2.75    1.   ]
 [  0.125   0.     -0.125   0.   ]]


In [ ]:
l = np.matrix(lu.L)
u = np.matrix(lu.U)
l * u

matrix([[-3., -5., 14., 13.],
        [-2., -4.,  5., 10.],
        [ 1.,  2., -2., -2.],
        [ 1.,  2., -2.,  6.]])

In [ ]:
A = np.matrix(A)
A * np.matrix(lu.inverse())

matrix([[ 3.55271368e-15,  1.00000000e+00, -1.42108547e-14,
          1.77635684e-15],
        [ 2.22044605e-16, -6.66666667e-01, -1.55431223e-15,
          1.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
          5.00000000e-01],
        [ 1.00000000e+00,  0.00000000e+00, -1.00000000e+00,
          0.00000000e+00]])

#### 2. Реализовать метод прогонки в виде программы, задавая в качестве входных данных ненулевые элементы матрицы системы и вектор правых частей. Используя разработанное программное обеспечение, решить СЛАУ с трехдиагональной матрицей.

In [ ]:
def tridiagonal_algo(A=None, alpha=None, beta=None, gamma=None, b=None):
    if not all([alpha, beta, gamma]) and A is not None:
        alpha = [0] + [A[i+1][i] for i in range(len(A)-1)]
        beta = [-A[i][i] for i in range(len(A))]
        gamma = [A[i][i+1] for i in range(len(A)-1)] + [0]
    P = [gamma[0] / beta[0]]
    Q = [-b[0] / beta[0]]
    for i in range(1, len(b)-1):
        P.append(gamma[i] / (beta[i] - alpha[i] * P[i-1]))
        Q.append((alpha[i] * Q[i-1] - b[i]) / (beta[i] - alpha[i] * P[i-1]))
    x = [(alpha[-1] * Q[-1] - b[-1]) / (beta[-1] - alpha[-1] * P[-1])]
    for i in range(len(b)-2, -1, -1):
        x.append(P[i] * x[-1] + Q[i])
    return x[::-1]

In [ ]:
A = np.array([[-11,  -9,  0,   0, 0],
              [  5, -15, -2,   0, 0],
              [  0,  -8, 11,  -3, 0],
              [  0,   0,  6, -15, 4],
              [  0,   0,  0,   3, 6]], dtype=float)
b = np.array([-122, -48, -14, -50, 42], dtype=float)
print_system(A, b)

{ -11x₁ - 9x₂  = -122.0
{ 5x₁ - 15x₂ - 2x₃  = -48.0
{ -8x₂ + 11x₃ - 3x₄  = -14.0
{ 6x₃ - 15x₄ + 4x₅  = -50.0
{ 3x₄ + 6x₅  = 42.0


In [ ]:
print('Решение системы: ', end='')
print_x_solution(tridiagonal_algo(A=A, b=b))

Решение системы: x₁ = 7;  x₂ = 5;  x₃ = 4;  x₄ = 6;  x₅ = 4;  


#### 5. Реализовать алгоритм QR – разложения матриц в виде программы. На его основе разработать программу, реализующую QR – алгоритм решения полной проблемы собственных значений произвольных матриц, задавая в качестве входных данных матрицу и точность вычислений. С использованием разработанного программного обеспечения найти собственные значения матрицы.

In [ ]:
def sign(x): return -1 if x < 0 else 0 if x == 0 else 1

def norm_2(x): return sqrt(sum([el**2 for el in x]))

In [ ]:
def QR(A):
    n = len(A)
    A = np.matrix(A)
    Q = np.matrix(np.eye(n))
    for i in range(n - 1):
        v = [0] * n
        v[i] = A[i, i] + sign(A[i, i]) * norm_2(np.squeeze(np.asarray(A[i:, i])))
        v[i+1:] = np.asarray(A[i+1:, i]).T[0]
        v = np.matrix(v).T
        H = np.matrix(np.eye(n) - 2 * v * v.T / norm_2(np.asarray(v)) ** 2)
        A = H * A
        Q *= H
    return Q, A

In [ ]:
def eigenvals_QR(A, e = 0.001):
    def ColumnError(A, i): return norm_2(np.asarray(A[i+1:, i]))
    def solveQuadEql(a, b, c):
        d = b**2 - 4 * a * c
        if d < 0:
            return (-b / 2*a, sqrt(-d) / 2*a), (-b / 2*a, -sqrt(-d) / 2*a)
        elif d == 0:
            return (-b / 2*a, 0), (-b / 2*a, 0)
        else:
            return ((-b + sqrt(d)) / 2*a, 0), ((-b - sqrt(d)) / 2*a, 0)
    def BlockError(x1, x2, r1, r2):
        r1, r2 = np.array(r1), np.array(r2)
        return max(norm_2(r1 - x1), norm_2(r2 - x2))

    n = len(A)
    x_Real = np.array([0] * n, dtype=float)
    x_Comp = np.array([[[42, 42]] * 2] * n, dtype=float)
    ok_Real = [False] * n
    ok_Comp = [False] * n
    ok_Real[-1] = True
    while True:
        Q, R = QR(A)
        A = R * Q
        for j in range(n - 1):
            if ColumnError(A, j) < e:
                x_Real[j] = A[j, j]
                ok_Real[j] = True
            r1, r2 = solveQuadEql(a = 1,
                                  b = -A[j+1, j+1]-A[j, j],
                                  c = A[j, j] * A[j+1, j+1] - A[j, j+1] * A[j+1, j])
            if BlockError(*x_Comp[j], r1, r2) < e:
                ok_Comp[j], ok_Comp[j+1] = True, True
            x_Comp[j] = r1, r2
        if all(any(ready) for ready in list(zip(ok_Real, ok_Comp))):
            break
    x = np.array([[0, 0]] * n, dtype=float)
    i = 0
    while i < n:
        if ok_Comp[i]:
            x[i], x[i+1] = x_Comp[i]
            i += 2
        else:
            x[i] = x_Real[i], 0
            i += 1
    return x

In [ ]:
A = np.array([[ 3, -7, -1],
              [-9, -8,  7],
              [ 5,  2,  2]], dtype=float)

In [ ]:
A = np.array([[ 3, -7, -1, 4, 2],
              [-9, -8,  7, -5, 1],
              [ 5,  -2,  2, 0, 3],
              [4, 2, -6, 4, 2],
              [1, 4, -9, 2, 4]], dtype=float)

In [ ]:
A = np.array([[0, 4, -2, -1, 7],
              [3, 1, -2, -6, -5],
              [-1, 2, -9, -5, -13],
              [-5, 2, 5, 16, -4],
              [2, 4, -1, -8, -8]], dtype=float)

In [ ]:
print_comp_solution(eigenvals_QR(A), 'λ')

λ₁ = 18.1267;
λ₂ = -1.3476;
λ₃ = -7.8193 + 4.1901 i;
λ₄ = -7.8193 - 4.1901 i;
λ₅ = 0.0;



#### 3. Реализовать метод простых итераций и метод Зейделя в виде программ, задавая в качестве входных данных матрицу системы, вектор правых частей и точность вычислений. Используя разработанное программное обеспечение, решить СЛАУ. Проанализировать количество итераций, необходимое для достижения заданной точности.

In [ ]:
def norm_c(a):
    a = np.asarray(a).T
    if hasattr(a[0], '__iter__'):
        a = [sum(abs(ai)) for ai in a]
    return max(a)

In [ ]:
A = np.array([[19, -4,  -9, -1],
              [-2, 20,  -2, -7],
              [ 6, -5, -25,  9],
              [ 0, -3,  -9, 12]], dtype=float)
b = np.array([100, -5, 34, 69], dtype=float)
print_system(A, b)

{ 19x₁ - 4x₂ - 9x₃ - 1x₄  = 100.0
{ -2x₁ + 20x₂ - 2x₃ - 7x₄  = -5.0
{ 6x₁ - 5x₂ - 25x₃ + 9x₄  = 34.0
{ -3x₂ - 9x₃ + 12x₄  = 69.0


##### Метод простых итераций

In [ ]:
def simple_iteration(A, b, e = 0.001):
    n = len(b)
    b = np.matrix([b[i] / A[i][i] for i in range(n)]).T
    A = np.matrix([[-A[i][j]/A[i][i] if i != j else 0 for j in range(n)] for i in range(n)])
    if max(norm_2(val) for val in eigenvals_QR(A)) >= 1:
        return None
    x = b.copy()
    normA = norm_c(A)
    if normA < 1:
        estim = (log10(e) - log10(norm_c(b)) + log10(1 - norm_c(A))) / log10(norm_c(A))
        err_koef = normA / (1 - normA)
    else:
        estim = None
        err_koef = 1
    iter = 0
    while True:
        iter += 1
        x_prev = x.copy()
        x = A * x + b
        if norm_c(x - x_prev) * err_koef < e:
            break
    return x, estim, iter

In [ ]:
x, estim, iter = simple_iteration(A, b, 0.1)
print('Априорная оценка числа итераций:', estim if estim else 'достаточное условие (||alpha|| < 1) не выполнено')
print('Фактическое число итераций:', iter)
print('Решение системы: ', end='')
print_x_solution(np.squeeze(x.tolist()))

Априорная оценка числа итераций: достаточное условие (||alpha|| < 1) не выполнено
Фактическое число итераций: 15
Решение системы: x₁ = 7.987;  x₂ = 4.001;  x₃ = 3.006;  x₄ = 8.98;  


##### Метод Зейделя

In [ ]:
def upp_triang(A):
    return [[A[i][j] if i <= j else 0 for j in range(len(A))] for i in range(len(A))]

In [ ]:
def seidel(A, b, e = 0.001):
    n = len(b)
    b = np.array([b[i] / A[i][i] for i in range(n)])
    A = np.array([[-A[i][j]/A[i][i] if i != j else 0 for j in range(n)] for i in range(n)])
    if max(norm_2(val) for val in eigenvals_QR(A)) >= 1:
        return None
    x = b.copy()
    normA = norm_c(A)
    err_koef = norm_c(upp_triang(A)) / (1 - normA) if normA < 1 else 1
    iter = 0
    while True:
        iter += 1
        x_prev = x.copy()
        x = []
        for i in range(n):
            xi = 0
            for j in range(len(x)):
                xi += A[i][j] * x[j]
            for j in range(len(x), n):
                xi += A[i][j] * x_prev[j]
            x.append(xi + b[i])
        x = np.array(x)
        if norm_c(x - x_prev) * err_koef < e:
            break
    return x, iter

In [ ]:
x, iter = seidel(A, b, 0.1)
print('Фактическое число итераций:', iter)
print('Решение системы: ', end='')
print_x_solution(x)

Фактическое число итераций: 6
Решение системы: x₁ = 7.948;  x₂ = 3.963;  x₃ = 2.969;  x₄ = 8.968;  


#### 4. Реализовать метод вращений в виде программы, задавая в качестве входных данных матрицу и точность вычислений. Используя разработанное программное обеспечение, найти собственные значения и собственные векторы симметрических матриц. Проанализировать зависимость погрешности вычислений от числа итераций.

In [ ]:
def Jacobi_rotation(A, e = 0.001):
    A = np.matrix(A)
    n = len(A)
    U_ret = np.matrix(np.eye(n))
    iter = 0
    while True:
        iter += 1
        mi, mj, mmax = 0, 0, 0
        for i in range(n):
            for j in range(i + 1, n):
                if mmax < abs(A[i, j]):
                    mmax = abs(A[i, j])
                    mi, mj = i, j
        fi = atan(2 * A[mi, mj] / (A[mi, mi] - A[mj, mj])) / 2 if A[mi, mi] != A[mj, mj] else pi / 4
        U = np.matrix(np.eye(n))
        U[mi, mi], U[mi, mj] = cos(fi), -sin(fi)
        U[mj, mi], U[mj, mj] = sin(fi),  cos(fi)
        U_ret *= U
        A = U.T * A * U
        t = 0
        for i in range(n):
            for j in range(i + 1, n):
                t += A[i, j] * A[i, j]
        if t < e**2: break
    return np.asarray(A.diagonal())[0], np.asarray(U_ret.T), iter

In [ ]:
A = np.array([[-7,  4,  5],
              [ 4, -6, -9],
              [ 5, -9, -8]])

In [ ]:
ei_val, ei_vec, iter = Jacobi_rotation(A, 0.3)
print('Фактическое число итераций:', iter)
print('Собственные значения: ', end='')
print_x_solution(ei_val, 'λ')
print('Собственные векторы:')
print_vec_solution(ei_vec)

Фактическое число итераций: 3
Собственные значения: λ₁ = -3.711;  λ₂ = 2.073;  λ₃ = -19.362;  
Собственные векторы:
x₁ = [0.88692126 0.34650007 0.30546422];
x₂ = [-0.0483791   0.7273351  -0.68457513];
x₃ = [-0.45938018  0.59238615  0.66185232];

